In [1]:
import time
import numpy as np
import scipy
from numpy import *
import matplotlib.pylab as plt
import operator
import pyscf
from pyscf import gto, scf, dft, tddft, data
import argparse

In [2]:
# parser = argparse.ArgumentParser(description='Davidson')

# parser.add_argument('-c', '--filename',       type=str, default='methanol.xyz', help='input filename')
# parser.add_argument('-m', '--method',         type=str, default='RKS', help='RHF RKS UHF UKS')
# parser.add_argument('-f', '--functional',     type=str, default='b3lyp', help='xc functional')
# parser.add_argument('-b', '--basis_set',      type=str, default='def2-SVP', help='basis sets')
# parser.add_argument('-i', '--initial_guess',  type=str, default='sTDA_A', help='initial_guess: diag_A or sTDA_A')
# parser.add_argument('-p', '--preconditioner', type=str, default='sTDA_A', help='preconditioner: diag_A or sTDA_A')
# parser.add_argument('-t', '--tolerance',      type=float, default='1e-5', help='residual norm convergence threshold')
# parser.add_argument('-n', '--nstates',        type=int, default='4', help='number of excited states')

# args = parser.parse_args()
# ################################################
# # read xyz file and delete its first two lines
# f = open(args.filename)
# coordinates = f.readlines()
# del coordinates[:2]
# ################################################

# print ('args.method =', args.method)
# ###########################################################################
# # build geometry in PySCF
# mol = gto.Mole()
# print (2)
# mol.build(atom = coordinates, basis = args.basis_set, symmetry = True)
# print (3)
# ###########################################################################

In [3]:
#np.set_printoptions (linewidth=300)

elements = ['H' , 'He', 'Li', 'Be', 'B' , 'C' , 'N' , 'O' , 'F' , 'Ne',
    'Na', 'Mg', 'Al', 'Si', 'P' , 'S' , 'Cl', 'Ar', 'K' , 'Ca',
    'Sc', 'Ti', 'V' , 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
    'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y' , 'Zr',
    'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn',
    'Sb', 'Te', 'I' , 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd',
    'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb',
    'Lu', 'Hf', 'Ta', 'W' , 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
    'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th',
    'Pa', 'U' , 'Np', 'Pu'] 
hardness = [
0.47259288,
0.92203391,
0.17452888,
0.25700733,
0.33949086,
0.42195412,
0.50438193,
0.58691863,
0.66931351,
0.75191607,
0.17964105,
0.22157276,
0.26348578,
0.30539645,
0.34734014,
0.38924725,
0.43115670,
0.47308269,
0.17105469,
0.20276244,
0.21007322,
0.21739647,
0.22471039,
0.23201501,
0.23933969,
0.24665638,
0.25398255,
0.26128863,
0.26859476,
0.27592565,
0.30762999,
0.33931580,
0.37235985,
0.40273549,
0.43445776,
0.46611708,
0.15585079,
0.18649324,
0.19356210,
0.20063311,
0.20770522,
0.21477254,
0.22184614,
0.22891872,
0.23598621,
0.24305612,
0.25013018,
0.25719937,
0.28784780,
0.31848673,
0.34912431,
0.37976593,
0.41040808,
0.44105777,
0.05019332,
0.06762570,
0.08504445,
0.10247736,
0.11991105,
0.13732772,
0.15476297,
0.17218265,
0.18961288,
0.20704760,
0.22446752,
0.24189645,
0.25932503,
0.27676094,
0.29418231,
0.31159587,
0.32902274,
0.34592298,
0.36388048,
0.38130586,
0.39877476,
0.41614298,
0.43364510,
0.45104014,
0.46848986,
0.48584550,
0.12526730,
0.14268677,
0.16011615,
0.17755889,
0.19497557,
0.21240778,
0.07263525,
0.09422158,
0.09920295,
0.10418621,
0.14235633,
0.16394294,
0.18551941,
0.22370139]
#in Hartree
HARDNESS = dict(zip(elements,hardness))

In [4]:
mol = gto.Mole()
mol.build(atom = '\
C         -4.89126        3.29770        0.00029;\
O         -3.49307        3.28429       -0.00328;\
H         -5.28213        2.58374        0.75736;\
H         -5.28213        3.05494       -1.01161;\
H         -5.23998        4.31540        0.27138;\
H         -3.22959        2.35981       -0.24953',\
basis = 'def2-SVP')


In [5]:
# mol = gto.Mole()
# mol.build(atom = '\
# C         -3.15617        2.59898        0.79547;\
# C         -1.79169        2.11570        0.42917;\
# O         -0.80893        2.56621        0.99508;\
# H         -1.66947        1.36193       -0.34183;\
# H         -3.35300        2.38970        1.86780;\
# H         -3.91803        2.07820        0.17854;\
# H         -3.22824        3.69190        0.61449',\
# basis = 'def2-SVP')

In [6]:
# mol = gto.Mole()
# mol.build(atom = 'O         -4.89126        3.29770        0.00029;\
# H         -3.49307        3.28429       -0.00328;\
# H         -5.28213        2.58374        0.75736', basis = 'def2-SVP', symmetry = True)

In [7]:
# mol = gto.Mole()
# mol.build(atom = '\
# C         -1.44673        2.80824       -0.07813;\
# O         -1.78998        3.80792       -0.69188;\
# N         -1.18291        1.66325       -0.74926;\
# N         -1.33221        2.85136        1.26946;\
# H         -1.53168        3.72026        1.78651;\
# H         -1.04365        2.01512        1.79805;\
# H         -1.26824        1.62382       -1.77554;\
# H         -0.89270        0.81388       -0.24298'\
# , basis = 'def2-SVP', symmetry = True)

In [8]:
# mf = dft.RKS(mol) 
# mf.conv_tol = 1e-12
# mf.grids.level = 9     # 0-9, big number for large mesh grids, default is 3
# mf.xc = 'b3lyp'
# mf.kernel()  #single point energy

In [9]:
# mf = dft.RKS(mol) 
# mf.conv_tol = 1e-12
# mf.grids.level = 9     # 0-9, big number for large mesh grids, default is 3
# mf.xc = 'cam-b3lyp'
# mf.kernel()  #single point energy

#ORCA: FINAL SINGLE POINT ENERGY = -115.576160742154
#Turbomole: total energy      =    -115.57615989622

In [10]:
mf = scf.RHF(mol) 
mf.conv_tol = 1e-13
mf.kernel()  #single point energy

converged SCF energy = -114.950987102225


-114.95098710222453

In [11]:
# CIS = pyscf.tdscf.rhf.CIS(mf)
# CIS.nstates = 5
# CIS.kernel()

In [12]:
# td = tddft.TDA(mf)
# start = time.time()
# td.kernel()    #compute first few excited states.
# end = time.time()
# print ('Pyscf time =', round(end-start,4))

In [13]:
#mf.analyze()
##MO energies

In [14]:
#mf.mulliken_pop_meta_lowdin_ao()
#population analysis
#mf.mulliken_pop()

In [15]:
#check whether a is symmetric
# def check_symmetric(a, tol=1e-12):
#     return np.all(np.abs(a-a.T) < tol)

In [16]:
def matrix_power (S,a):
    s,ket = np.linalg.eigh(S)
    # S = mf.get_ovlp() #.get_ovlp() is basis overlap matrix
    # S = np.dot(np.linalg.inv(c.T), np.linalg.inv(c))
    # #s are eigenvalues, must be all positive
    # #each column of ket is a eigenket
    s = s**a
    X = np.linalg.multi_dot([ket,np.diag(s),ket.T])
    #X == S^1/2
    return X

def orthonormalize (C):
    X = matrix_power(mf.get_ovlp(), 0.5)
    # X = S^1/2
    C = np.dot(X,C)
    return C
# C is orthonormalized coefficient matrix
# np.dot(C.T,C) is a identity matrix

def coefficient_matrix ():
    C = mf.mo_coeff
    # mf.mo_coeff is the coefficient matrix
    C = orthonormalize (C)
    return C
# rthogonalized MO coefficients 
C = coefficient_matrix ()

In [17]:
total_begin = time.time()

In [18]:
Natm = mol.natm
MOe = mf.mo_energy  
#an array of MO energies, in Hartree

occupied = len(np.where(mf.mo_occ > 0)[0])
#mf.mo_occ is an array of occupance [2,2,2,2,2,0,0,0,0.....]
virtual = len(np.where(mf.mo_occ == 0)[0])

AO = [int(i.split(' ',1)[0]) for i in mol.ao_labels()] 
# .split(' ',1) is to split each element by space, split once.
# mol.ao_labels() it is Labels of AO basis functions, AO is a list of corresponding atom_id
# AO == [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2]
N_bf = len(AO)

a_x = 0.38
beta1= 1.86
beta2=0
alpha1= 0.9
alpha2=0
beta = beta1 + beta2 * a_x
alpha = alpha1 + alpha2 * a_x

In [19]:
#build gammaJ and gammaK matrix

#function to return chemical hardness from dictionary HARDNESS
def Hardness (atom_id):
    atom = mol.atom_pure_symbol(atom_id) 
    return HARDNESS[atom]
# mol.atom_pure_symbol(atom_id) returns pure element symbol, no special characters
def eta (atom_A_id, atom_B_id):
    eta = (Hardness(atom_A_id) + Hardness(atom_B_id))/2
    return eta
R = pyscf.gto.mole.inter_distance(mol, coords=None) 
#Inter-particle distance array
# unit == ’Bohr’, Its value is 5.29177210903(80)×10^(−11) m

In [20]:
start = time.time()
GammaJ = np.zeros([Natm, Natm])
for i in range (0, Natm):
    for j in range (0, Natm):
        GammaJ[i,j] = (R[i, j]**beta + (a_x * eta(i, j))**(-beta))**(-1/beta)

GammaK = np.zeros([Natm, Natm])
for i in range (0, Natm):
    for j in range (0, Natm):
        GammaK[i,j] = (R[i, j]**alpha + eta(i, j)**(-alpha)) **(-1/alpha)

In [21]:
Natm = mol.natm 
def generateQ ():
    q = np.zeros([Natm, N_bf, N_bf])
    #N_bf is number Atomic orbitals, occupied+virtual, q is same size with C
    C = coefficient_matrix ()
    for atom_id in range (0, Natm):
        for i in range (0, N_bf):
            for p in range (0, N_bf):
                for mu in range (0, N_bf):
                    if AO[mu] == atom_id:
                        #collect all basis functions centered on atom_id
                        # the last loop is to sum up all C_mui*C_mup, calculate element q[i,p]
                        q[atom_id,i,p] += C[mu,i]*C[mu,p]
                        #q[i,p] += 2*C[i,mu]*C[p,mu]
    return q

In [22]:
q_tensors   = generateQ    ()
q_tensor_ij = q_tensors [:, :occupied,:occupied]
q_tensor_ab = q_tensors [:, occupied:,occupied:]
q_tensor_ia = q_tensors [:, :occupied,occupied:]

Q_K = np.einsum('Bjb, AB -> Ajb', q_tensor_ia, GammaK)
Q_J = np.einsum('Bab, AB -> Aab', q_tensor_ab, GammaJ)

end = time.time()

Q_gamma_tensors_building_time = end - start
print ('Q_tensorQ_gamma_tensors_building_time =', Q_gamma_tensors_building_time)

Q_tensorQ_gamma_tensors_building_time = 0.158735990524292


In [23]:
#iajb_v = np.einsum('Aia, Bjb, AB, jbm -> iam', q_tensor_ia, q_tensor_ia, GammaK, v)
#ijab_v = np.einsum('Aij, Bab, AB, jbm -> iam', q_tensor_ij, q_tensor_ab, GammaJ, v)

def iajb_fly (V):
    V = V.reshape(occupied, virtual, -1)
    Q_K_V = np.einsum('Ajb, jbm -> Am', Q_K, V)
    iajb_V = np.einsum('Aia, Am -> iam', q_tensor_ia, Q_K_V).reshape(occupied*virtual, -1)
#     print('iajb done')
    return iajb_V

def ijab_fly (V):
    V = V.reshape(occupied, virtual, -1)
    ijab_V = np.einsum('Aij, Aab, jbm -> iam', q_tensor_ij, Q_J, V).reshape(occupied*virtual, -1)
#     print ('ijab done')
    return ijab_V

delta_diag_A = np.zeros((occupied, virtual))
for i in range (0, occupied):
    for a in range (0, virtual):
        delta_diag_A[i,a] = (MOe[occupied+a] - MOe[i])

# delta_ij = np.eye(occupied,occupied)
# delta_ab = np.eye(virtual,virtual)
def delta_fly (V):
    V = V.reshape(occupied, virtual, -1)
#     print ('Shape of V = ', np.shape(V))
    #delta_v = np.einsum('ij,ab,ia,jb -> ia',delta_ij,delta_ab,delta_diag_A, v)
    delta_v = np.einsum('ia,iam -> iam', delta_diag_A, V).reshape(occupied*virtual, -1)
    return delta_v

def sTDA_fly (V):
    V = V.reshape(occupied*virtual,-1)
    # -1 means shape of first dimension is not asigned, but can be inferred with the rest dimension
    # this feature can deal with multiple vectors
    sTDA_V =  delta_fly (V) + 2*iajb_fly (V) - ijab_fly (V) 
#     sTDA_V = sTDA_V.reshape(occupied*virtual, -1)
#     print (np.shape(sTDA_v))
    return sTDA_V

In [249]:
n = occupied * virtual
I = np.eye(n)

start = time.time()
sTDA_A = sTDA_fly (I)
end = time.time()  

sTDA_A_building_time = end - start

print ('Q_tensorQ_gamma_tensors_building_time =', Q_gamma_tensors_building_time)

print ('sTDA_A building time = ', sTDA_A_building_time)

total_end = time.time()  

total_build_time = total_end - total_begin

print ('total building sTDA A time =', total_build_time)

s,k = np.linalg.eigh(sTDA_A)
print (s[:4]*27.21138624598853)
# [12.2388045  13.59944497 13.79151538 15.04073066]

# plt.matshow(sTDA_A)
# plt.show()

Q_tensorQ_gamma_tensors_building_time = 0.158735990524292
sTDA_A building time =  0.2549092769622803
total building sTDA A time = 43539.75000190735
[12.2388045  13.59944497 13.79151538 15.04073066]


In [287]:
a = np.array([[3,8,6,8],[4,4,8,2]])
# b = np.arange(5,7)
# print (np.shape (b))
# c = a/b
# # print (c) 
# b = np.linalg.norm(a, axis=0, keepdims = True)
# print (b)
# c = a/b
# print (c)

# d = np.linalg.norm(c, axis=0, keepdims = True)
# print (d)

b = a[:, 2:]
print (b)

[[6 8]
 [8 2]]


In [177]:
def orthonormalize (v1, v2):
    v2 = v2 - (np.dot(v1, v2) / np.dot(v1, v1)) * v1
#     v2 = v2 - np.dot(v1, v2) * v1
    v2 = v2/np.linalg.norm(v2)
    return v2

In [178]:
def Gram_Schdmit (A):
    # A matrix has J columns, orthonormaize each columns
    
    J = np.shape(A)[1]
    
    A[:,0] = A[:,0]/np.linalg.norm(A[:,0])

    for p in range (0, J - 1):
        for q in range (p + 1, J):
            A[:, q] = orthonormalize(A[:, p], A[:, q])
    return A

In [179]:
# Q = np.random.rand(10,5)
# print (Q)
# #built-in function of np approaching, V is the a orthonnormalized matrix
# V,R = np.linalg.qr(Q)
# print (V)
# print (np.dot(V.T, V))

In [310]:
# Q = Gram_Schdmit (Q)
# print (Q)
# print (np.dot(Q.T, Q))


2

In [294]:
def Gram_Schdmit_append (A, B):
    # A_I*J, B_I*K
    # A is already orthonormalized
    # orthonormalize matrix B against A, meanwhile append B to A

    
    P = np.shape(A)[1]
    Q = np.shape(B)[1]
#     print ('K =', K)

#     B = B/np.linalg.norm(B, axis=0, keepdims = True)
    #normalize each vector in B first
    
    C = np.append(A, B, axis = 1)
    
    for p in range (0, P + Q -1):
        for q in range (max(P, p+1), P + Q):
            C[:, q] = orthonormalize(C[:, p], C[:, q])
#     C = C/np.linalg.norm(C, axis=0, keepdims = True)

    
    return C

In [295]:
# a = np.random.random ((9,4))
# a = Gram_Schdmit (a)
# b = np.random.random ((9,4))
# print (a)
# print (b)

In [296]:
# c = Gram_Schdmit_append (a, b)
# print (c)
# print ('c.T c =', np.dot(c.T,c))

# d = np.append (a,b, axis=1)
# e = Gram_Schdmit (d)

# print ('e.T e =', np.dot(e.T,e))
# print (e)

# T,U = np.linalg.qr(d)
# print ('standard')
# print ('T = ', T)
# print ('d.T d =', np.dot(d.T,d))

In [336]:
###########################################################################################
def preconditioner_sTDA (B, eigen_lambda, Y=0): 
    # (sTDA_A - eigen_lambda*I)^-1 B = X 
    # AX - X\lambda = B

    # columns of B are the vectors to be preconditioned, 
    N_rows = np.shape(B)[0]
    
    B = B.reshape(n,-1)

    N_vectors = np.shape(B)[1]
    #number of vectors to be preconditioned
    
    bnorm = np.linalg.norm(B, axis=0, keepdims = True)
    #norm of each vectors in B, shape (1,-1)
    B = B/bnorm
#     print ('shape of B=', np.shape(B))
    start = time.time()
    tol = 1e-5     # Convergence tolerance
    max = 30      # Maximum number of iterations  
    
    diag = delta_diag_A.flatten()
#     # delta_diag_A.flatten() is (\spsilon_a-\spsilon_i)

    D = np.zeros((N_rows,N_vectors))
    for i in range (0, N_vectors):
        D[:,i] = diag - eigen_lambda[i]
#         D[:,i][(D[:,i]<1e-16)&(D[:,i]>=0)] = 1e-20
#         D[:,i][(D[:,i]>-1e-16)&(D[:,i]<0)] = -1e-20
    #D is preconditioner for each state 
    print ('shape of D =', np.shape(D))
    ###########################################
    #initial guess: (diag(A) - \lambda)^-1 B.
    V = B/D
    V = Gram_Schdmit(V)
    print ('shape of V =', np.shape(V))
    
    W = sTDA_fly(V) 
    #V is guess holder, 
    # at the end of each iteration, V will be appended with new guess vectors
    
    Lambda = np.diag(eigen_lambda)

    ####################################################################################
    # Begin iterations
    for i in range (0, max):
        print ('Iteration =', i)
        sub_B = np.dot(V.T, B) 
        sub_A = np.dot(V.T, W)    
        #project sTDA_A matrix and vector B into subspace 
        m = np.shape(sub_A)[0]
        print ('subaspace size = ', np.shape(sub_A))
        print ('check V', np.linalg.norm(np.dot(V.T, V)- np.eye(m)))
        #m is always the size of subspace
#         print ('Size of subspace =', m)
        # size of subspace
        sub_guess = scipy.linalg.solve_sylvester(sub_A, - Lambda, sub_B)   
        #scipy.linalg.solve_sylvester(A,B,Q) # solve equation AX + XB = Q
#         print ('shape of sub_guess = ', np.shape(sub_guess))
#         print ('shape of V = ', np.shape(V))
       
        full_guess = np.dot(V, sub_guess)
#         print ('shape of full_guess = ', np.shape(full_guess))
#         print ('shape of sTDA_fly(full_guess) = ', np.shape(sTDA_fly(full_guess)))
#         print ('shape of eigen_lambda * full_guess = ', np.shape(eigen_lambda * full_guess))       
        
        residual = sTDA_fly(full_guess) - full_guess*eigen_lambda - B  
        
#         print ('shape of residual rrrrrrr= ', np.shape(residual))   
        Norms_of_residual = np.linalg.norm (residual, axis=0, keepdims = True)
        print ('Norms_of_residual =', Norms_of_residual)

#         print ('shape of Norm =', np.shape(Norms_of_residual))
        
        max_norm = np.max(Norms_of_residual)
        
        print ('max_norm = ', max_norm)
        if max_norm < tol:
            break
#         index = np.where(max_norm < tol)
#         D[:, index] = 1
        guess = residual/D 
        #preconditioning step

#         V = Gram_Schdmit_append(V, guess)
        V = np.append(V, guess, axis = 1)
        V = Gram_Schdmit(V)
#         W = np.append(W, sTDA_fly(guess), axis=1)
        W = sTDA_fly(V)
        # add new guess to the guess space
    ####################################################################################      

    #########################################################################################
#    print ('sTDA_Iteration steps =', i)
    end = time.time()
    print ('sTDA_Precondition time:', round(end-start,4))

# del list[n]
# A = np.delete(A, n, axis=1)

    return (full_guess*bnorm)
#     return (full_guess)
###########################################################################################

In [339]:
n_residuals = 10

B = np.random.random((occupied*virtual, n_residuals))

Y = None

eigen_lambda = np.arange(10, 10+n_residuals)

In [340]:
preconditioned_B =  preconditioner_sTDA (B, eigen_lambda, Y)
print ('Norm of preconditioned_B', np.linalg.norm(preconditioned_B))

n = np.shape(sTDA_A)[0]
true_answer = scipy.linalg.solve_sylvester(sTDA_A, -np.diag(eigen_lambda), B)

print ('Norm of true_answer     ',np.linalg.norm(true_answer))
print (np.shape(true_answer))

print ('Norm of (preconditioned_B - true_answer): ', np.linalg.norm(true_answer - preconditioned_B))
print (np.shape(true_answer - preconditioned_B))

preconditioned_B = preconditioned_B/np.linalg.norm(preconditioned_B)
true_answer = true_answer/np.linalg.norm(true_answer)
print ('Norm of (normalized_preconditioned_B - normalized_true_answer): ', np.linalg.norm(true_answer - preconditioned_B))

shape of D = (351, 10)
shape of V = (351, 10)
Iteration = 0
subaspace size =  (10, 10)
check V 1.6117837671454163e-15
Norms_of_residual = [[0.0502539  0.71425581 1.54609148 5.92280838 1.26422076 7.43246847
  0.03609005 0.03719656 0.067809   1.84547089]]
max_norm =  7.43246846887748
Iteration = 1
subaspace size =  (20, 20)
check V 2.093048558544891e-14
Norms_of_residual = [[5.46360380e-03 2.83412503e-01 1.35473821e+01 3.95514282e+00
  6.34262308e-01 1.69695568e-01 7.00003120e-03 1.02744608e-03
  7.51227599e-04 5.17025744e-03]]
max_norm =  13.547382139597262
Iteration = 2
subaspace size =  (30, 30)
check V 4.587827368702654e-15
Norms_of_residual = [[3.81775847e-04 1.69050331e-02 3.36908376e+00 9.26060472e+00
  6.59667525e-01 2.76127918e-02 2.85633959e-04 2.14551681e-05
  5.61033263e-05 4.64007881e-04]]
max_norm =  9.260604718887572
Iteration = 3
subaspace size =  (40, 40)
check V 6.336743671823073e-15
Norms_of_residual = [[8.63036984e-05 1.59467588e-02 4.53764870e+00 1.72423022e+00
  1.0

In [322]:
a = np.array([[1,2],
                 [3,4]])
b = np.array([[5,6]])
c = b.reshape(2,1)
print (a/b)
print (a/c)

[[0.2        0.33333333]
 [0.6        0.66666667]]
[[0.2        0.4       ]
 [0.5        0.66666667]]


In [332]:
###########################################################################################
def preconditioner_sTDA (B, eigen_lambda, Y=0): 
    # (sTDA_A - eigen_lambda*I)^-1 B = X 
    # AX - X\lambda = B

    # columns of B are the vectors to be preconditioned, 
    n = np.shape(B)[0]
    
    B = B.reshape(n,-1)

    N_vectors = np.shape(B)[1]
    #number of vectors to be preconditioned
    
    bnorm = np.linalg.norm(B, axis=0, keepdims = True)
    #norm of each vectors in B, shape (1,-1)
    B = B/bnorm
#     print ('shape of B=', np.shape(B))
    start = time.time()
    tol = 1e-5     # Convergence tolerance
    max = 30      # Maximum number of iterations  
    
    diag = delta_diag_A.flatten()
#     # delta_diag_A.flatten() is (\spsilon_a-\spsilon_i)

    D = np.zeros([n,N_vectors])
    for i in range (0, N_vectors):
        D[:,i] = diag - eigen_lambda[i]
#         D[:,i][(D[:,i]<1e-16)&(D[:,i]>=0)] = 1e-20
#         D[:,i][(D[:,i]>-1e-16)&(D[:,i]<0)] = -1e-20
    #D is preconditioner for each state 
    print ('shape of D =', np.shape(D))
    ###########################################
    #initial guess: (diag(A) - \lambda)^-1 B.
    V = B/D
    V = Gram_Schdmit(V)
    print ('shape of V =', np.shape(V))
    
    W = sTDA_fly(V) 
    #V is guess holder, 
    # at the end of each iteration, V will be appended with new guess vectors
    
    Lambda = np.diag(eigen_lambda)

    ####################################################################################
    # Begin iterations
    for i in range (0, max):
        print ('Iteration =', i)
        sub_B = np.dot(V.T, B) 
        sub_A = np.dot(V.T, W)    
        #project sTDA_A matrix and vector B into subspace 
        m = np.shape(sub_A)[0]
        print ('subaspace size = ', np.shape(sub_A))
        print ('check V', np.linalg.norm(np.dot(V.T, V)- np.eye(m)))
        #m is always the size of subspace
#         print ('Size of subspace =', m)
        # size of subspace
        sub_guess = scipy.linalg.solve_sylvester(sub_A, - Lambda, sub_B)   
        #scipy.linalg.solve_sylvester(A,B,Q) # solve equation AX + XB = Q
#         print ('shape of sub_guess = ', np.shape(sub_guess))
#         print ('shape of V = ', np.shape(V))
       
        full_guess = np.dot(V, sub_guess)
#         print ('shape of full_guess = ', np.shape(full_guess))
#         print ('shape of sTDA_fly(full_guess) = ', np.shape(sTDA_fly(full_guess)))
#         print ('shape of eigen_lambda * full_guess = ', np.shape(eigen_lambda * full_guess))       
        
        residual = sTDA_fly(full_guess) - full_guess*eigen_lambda - B  
        
#         print ('shape of residual rrrrrrr= ', np.shape(residual))   
        Norms_of_residual = np.linalg.norm (residual, axis=0, keepdims = True)
        print ('Norms_of_residual =', Norms_of_residual)

#         print ('shape of Norm =', np.shape(Norms_of_residual))
        
        max_norm = np.max(Norms_of_residual)
        
        print ('max_norm = ', max_norm)
        if max_norm < tol:
            break
#         index = np.where(max_norm < tol)
#         D[:, index] = 1
        guess = residual/D 
        #preconditioning step

#         V = Gram_Schdmit_append(V, guess)
        V = np.append(V, guess, axis = 1)
        V = Gram_Schdmit(V)
#         W = np.append(W, sTDA_fly(guess), axis=1)
        W = sTDA_fly(V)
        # add new guess to the guess space
    ####################################################################################      

    #########################################################################################
#    print ('sTDA_Iteration steps =', i)
    end = time.time()
    print ('sTDA_Precondition time:', round(end-start,4))

# del list[n]
# A = np.delete(A, n, axis=1)

    return (full_guess*bnorm)
#     return (full_guess)
###########################################################################################

In [ ]:
td = tddft.TDA(mf)
vind, hdiag = td.gen_vind(mf)
print (type(hdiag))
print (np.shape(hdiag))

In [ ]:
######################################################################################
def A_diag_initial_guess (k):
    m = min([2*k, k+8, occupied*virtual])
    # m is size of subspace Hamiltonian, amount of initial guesses   
    # m=k works for H2, m=4k works for H2O
    V = np.zeros((n, m)) 
    #array of zeros, a container to hold current guess vectors 
    W = np.zeros((n, m)) 
    
    sort = hdiag.argsort()
    for j in range(0,m):
        V[int(np.argwhere(sort == j)), j] = 1   
        # positions with lowest values set as 1
        W[:, j] = vind(V[:, j])
    # W = Av, create transformed guess vectors

    return (m, V, W)

def sTDA_initial_guess (k):
    m = min([2*k, k+8, occupied*virtual])
    # m is size of subspace Hamiltonian, amount of initial guesses   
    # m=k works for H2, m=4k works for H2O
    V = np.zeros((n, m)) 
    # array of zeros, a container to hold current guess vectors, v
    W = np.zeros((n, m)) 
    # a container to hold transformed guess vectors, Av
    eigvalues, eigkets = np.linalg.eigh(sTDA_A)
    # eigv, eigk = Davidson (A, m, 1e-5, default_initial_guess, A_diag_preconditioner)
    #!!!!!!!! diagonalize sTDA_A amtrix
    for j in range(0,m):
        V[:, j] = eigkets [:, j]
        W[:, j] = vind(V[:, j])
        
    return (m, V, W)
######################################################################################



###################################################################
def A_diag_preconditioner (residual, sub_eigenvalue):
    d = hdiag - sub_eigenvalue
    d[(d<1e-16)&(d>=0)] = 1e-16
    d[(d>-1e-16)&(d<0)] = -1e-16   
    #kick out all small values
    
    new_vec = residual/d
    return new_vec

def sTDA_preconditioner (residual, sub_eigenvalue):
    new_vec = np.dot(np.linalg.inv(sTDA_A - sub_eigenvalue*I),residual)
    return new_vec
####################################################################

In [ ]:
################################################################################
# def Davidson (k, tol, initial_guess, preconditioner):

    if initial_guess == 'sTDA_initial_guess':
        initial_guess = sTDA_initial_guess
        print ('Initial guess: sTDA A matrix')
    elif initial_guess == 'A_diag_initial_guess':
        initial_guess = A_diag_initial_guess
        print ('Initial guess: Diagonal of Pseudo A matrix')
        
    if preconditioner == 'sTDA_fly_preconditioner':
        preconditioner = preconditioner_sTDA
        print ('Preconditioner: on-the-fly sTDA A matrix')
        
    elif preconditioner == 'full_sTDA_preconditioner':
        preconditioner = sTDA_preconditioner
        print ('Preconditioner: full sTDA A matrix')
        
    elif preconditioner == 'A_diag_preconditioner':
        preconditioner = A_diag_preconditioner
        print ('Preconditioner: Diagonal of Pseudo A matrix')
    start = time.time()
    
    #tol = 1e-5      
    # Convergence tolerance
    n = occupied*virtual
    max = 90      
    # Maximum number of iterations

    ########################################################################################### 
    for i in range(0, max):
        print ('Iteration = ', i)
        sum_convec = 0
        # total converged eigenvectors
        # breaf if sum_convec == k
        
        #################################################
        # generate initial guess
        if i == 0:
            m, V, W = initial_guess(k)
        print ('Initial guess done')
        #################################################

        sub_A = np.dot(V.T, W)  
        # sub_A is subspace A matrix
        sub_eigenvalue, sub_eigenket = np.linalg.eigh(sub_A) 
        # Diagonalize the subspace Hamiltonian, and sorted.

        lasit_newvec = 0
        # amount of new vectors added in last iteration, ranging from 1 to k
        # because not all new guess_vectors can survive the Gram-Schmidt

        
        eigen_lambda = sub_eigenvalue[:k]
        #diagonal elements are smallest k eigenvalues
        
        
        
        
        
        
        
        
        
        ####################################################################################
        for x in range(0,k):      
            #looking at first k vecrors one by one, check whether they are roots
            residual = np.dot((W[:,:m]- sub_eigenvalue[x]*V[:,:m].reshape(n,-1)), sub_eigenket[:,x])
            # residual = A (Vs) - lambda*(Vs) 
            # Vs np.dotV([:,:m])s[:,x]), projects the subspace eigenket back to full space
            
            #print ('Residual created')
        

            norm = np.linalg.norm(residual)
            if norm <= tol:
                sum_convec += 1
            else:
                # current guess is not good enough, 
                # so we use current guess to create new guess vectors
                #########################################################
                new_vec = preconditioner (residual, sub_eigenvalue[x])          
                #########################################################
                # preconditioner
                
                new_vec = new_vec/np.linalg.norm (new_vec) 
                new_vec = new_vec.reshape(-1,1)
                # normalize before Gram-Schmidt 
                for y in range (0, m + lasit_newvec):  
                    # orthonormalize the new vector against all old vectors
                    new_vec = new_vec - np.dot(V[:,y], new_vec) * V[:,y].reshape(-1,1)   
                    
                norm = np.linalg.norm (new_vec)
                if norm > 1e-16:
                    new_vec = new_vec/norm
                    # normalzie the new vector, now Gram-Schmidt is done

                    
                    V = np.append (V, new_vec, axis=1)
                    # put the new guess into container
                    
                    trans_new_vec = vind(new_vec)
                    #print ('Shape of trans_new_vec =', np.shape(trans_new_vec)) = (1,351)
                    W = np.append (W, trans_new_vec.T, axis = 1)
                    # put transformed guess Av into container
                    lasit_newvec += 1
        ####################################################################################      
        if sum_convec == k:
            break
        m += lasit_newvec
    ########################################################################################### 

    Eigenkets = np.dot(V[:,:m], sub_eigenket[:, :k])

    
    print ('Iteration steps =', i+1)
    print ('Final subspace size = ', np.shape(sub_A))
    #print ('Davidson time:', round(end-start,4))
    
    return (sub_eigenvalue[:k]*27.21138624598853, Eigenkets[:,:k])

In [ ]:
################################################################################
def Davidson (k, tol, initial_guess, preconditioner):

    if initial_guess == 'sTDA_initial_guess':
        initial_guess = sTDA_initial_guess
        print ('Initial guess: sTDA A matrix')
    elif initial_guess == 'A_diag_initial_guess':
        initial_guess = A_diag_initial_guess
        print ('Initial guess: Diagonal of Pseudo A matrix')
        
    if preconditioner == 'sTDA_fly_preconditioner':
        preconditioner = preconditioner_sTDA
        print ('Preconditioner: on-the-fly sTDA A matrix')
        
    elif preconditioner == 'full_sTDA_preconditioner':
        preconditioner = sTDA_preconditioner
        print ('Preconditioner: full sTDA A matrix')
        
    elif preconditioner == 'A_diag_preconditioner':
        preconditioner = A_diag_preconditioner
        print ('Preconditioner: Diagonal of Pseudo A matrix')
    start = time.time()
    
    #tol = 1e-5      
    # Convergence tolerance
    n = occupied*virtual
    max = 90      
    # Maximum number of iterations

    ########################################################################################### 
    for i in range(0, max):
        print ('Iteration = ', i)
        sum_convec = 0
        # total converged eigenvectors
        # breaf if sum_convec == k
        
        #################################################
        # generate initial guess
        if i == 0:
            m, V, W = initial_guess(k)
        print ('Initial guess done')
        #################################################

        sub_A = np.dot(V.T, W)  
        # sub_A is subspace A matrix
        sub_eigenvalue, sub_eigenket = np.linalg.eigh(sub_A) 
        # Diagonalize the subspace Hamiltonian, and sorted.

        lasit_newvec = 0
        # amount of new vectors added in last iteration, ranging from 1 to k
        # because not all new guess_vectors can survive the Gram-Schmidt

        
        ####################################################################################
        for x in range(0,k):      
            #looking at first k vecrors one by one, check whether they are roots
            residual = np.dot((W[:,:m]- sub_eigenvalue[x]*V[:,:m].reshape(n,-1)), sub_eigenket[:,x])
            # residual = A (Vs) - lambda*(Vs) 
            # Vs np.dotV([:,:m])s[:,x]), projects the subspace eigenket back to full space
            
            #print ('Residual created')
        

            norm = np.linalg.norm(residual)
            if norm <= tol:
                sum_convec += 1
            else:
                # current guess is not good enough, 
                # so we use current guess to create new guess vectors
                #########################################################
                new_vec = preconditioner (residual, sub_eigenvalue[x])          
                #########################################################
                # preconditioner
                
                new_vec = new_vec/np.linalg.norm (new_vec) 
                new_vec = new_vec.reshape(-1,1)
                # normalize before Gram-Schmidt 
                for y in range (0, m + lasit_newvec):  
                    # orthonormalize the new vector against all old vectors
                    new_vec = new_vec - np.dot(V[:,y], new_vec) * V[:,y].reshape(-1,1)   
                    
                norm = np.linalg.norm (new_vec)
                if norm > 1e-16:
                    new_vec = new_vec/norm
                    # normalzie the new vector, now Gram-Schmidt is done

                    
                    V = np.append (V, new_vec, axis=1)
                    # put the new guess into container
                    
                    trans_new_vec = vind(new_vec)
                    #print ('Shape of trans_new_vec =', np.shape(trans_new_vec)) = (1,351)
                    W = np.append (W, trans_new_vec.T, axis = 1)
                    # put transformed guess Av into container
                    lasit_newvec += 1
        ####################################################################################      
        if sum_convec == k:
            break
        m += lasit_newvec
    ########################################################################################### 

    Eigenkets = np.dot(V[:,:m], sub_eigenket[:, :k])

    
    print ('Iteration steps =', i+1)
    print ('Final subspace size = ', np.shape(sub_A))
    #print ('Davidson time:', round(end-start,4))
    
    return (sub_eigenvalue[:k]*27.21138624598853, Eigenkets[:,:k])

In [ ]:
print ('#################################################################')
print ('In-house Davdison codes:')

k = 40
tol = 1e-5
# initial_guess = 'sTDA_initial_guess'
# preconditioner = 'sTDA_fly_preconditioner'

print ('Number of excited states =', k)
print ('Residual convergenve threshold =', tol)

print ('#################################################################')

start = time.time()
Excitation_energies, kets = Davidson (k, tol, 'sTDA_initial_guess', 'sTDA_fly_preconditioner')
end = time.time()

print ('In-house Davidson time= ', end - start)
print ('Excited State energies (eV) =')
print (Excitation_energies)
print ('#################################################################')

In [ ]:
print ('PySCF TDA-TDDFT codes:')
start = time.time()
td.nstates = k
td.kernel()
end = time.time()
print ('PySCF Davidson time= ', end - start)
print ('#################################################################')

In [ ]:
print ('#################################################################')
start = time.time()
Excitation_energies, kets = Davidson (k, tol, 'A_diag_initial_guess', 'A_diag_preconditioner')
end = time.time()

print ('In-house Davidson time= ', end - start)
print ('Excited State energies (eV) =')
print (Excitation_energies)
print ('#################################################################')

In [ ]:
def generateQ (atom_id):
    q = np.zeros([N_bf, N_bf])
    #N_bf is number Atomic orbitals, q is same size with C
    
    C = coefficient_matrix ()
    for i in range (0, N_bf):
        for p in range (0, N_bf):
            for mu in range (0, N_bf):
                if AO[mu] == atom_id:
                    #collect all basis functions centered on atom_id
                    # the last loop is to sum up all C_mui*C_mup, calculate element q[i,p]
                    q[i,p] += C[mu,i]*C[mu,p]
                    #q[i,p] += 2*C[i,mu]*C[p,mu]
    return q

# home_made population analysis
# for atom_id in range (0, Natm):
#     print (check_symmetric(Qmatrix[atom_id], tol=1e-12))
#     m = 0
#     for i in range (0, occupied):
#         m += Qmatrix[atom_id][i,i]
#         #sum over occupied orbitals
#     print (m)


In [ ]:
start = time.time()
Qmatrix = [(generateQ(atom_id)) for atom_id in range (0, Natm)]
#a list of q matrix
end = time.time()

Qmatrix = np.asarray(Qmatrix)
print (end - start)
print (np.shape(Qmatrix))

In [ ]:
#define two electron intergeral (pq|rs)
Natm = mol.natm 
#number of atoms
Natm = mol.natm 

def ele_intJ (i,j,a,b):
    ijab = 0
    for atom_A_id in range (0, Natm):
        for atom_B_id in range (0, Natm):
            ijab += Qmatrix[atom_A_id,i,j] * Qmatrix[atom_B_id,a,b] * GammaJ[atom_A_id, atom_B_id]
    return ijab
        
def ele_intK (i,a,j,b):
    iajb = 0
    for atom_A_id in range (0, Natm):
        for atom_B_id in range (0, Natm):
            iajb += Qmatrix[atom_A_id,i,a] * Qmatrix[atom_B_id,j,b] * GammaK[atom_A_id, atom_B_id]
    return iajb

In [ ]:
#build A matrix
def build_sTDA_A ():
    A = np.zeros ([occupied*virtual, occupied*virtual])
    m = -1
    for i in range (0, occupied):
        for a in range (occupied, N_bf):
            m += 1 #for each ia pair, it corresponds to a certain row
            n = -1
            for j in range (0, occupied):
                for b in range (occupied, N_bf):
                    n += 1 #for each jb pair, it corresponds to a certain column        
                    if i==j and a==b:
                        A[m,n] =   2*ele_intK(i,a,j,b) - ele_intJ(i,j,a,b) + (MOe[a]-MOe[i]) 
                    else:
                        A[m,n] = 2*ele_intK(i,a,j,b) - ele_intJ(i,j,a,b)
    print ('sTDA_A matrix built, size =', np.shape(A)[0])
    return A

start = time.time()
sTDA_A = build_sTDA_A ()
end = time.time()
print ('A_sTDA building time =', round (end - start, 2))

In [ ]:
plt.matshow(sTDA_A)
plt.show()

In [ ]:
eigv,eigk = np.linalg.eigh(sTDA_A)
print (eigv[:5]*27.21138624598853)

In [ ]:
# print (check_symmetric(A, tol=1e-8))

In [ ]:
print (np.linalg.norm(sTDA_A - sTDA_fly(I)))

In [ ]:
# Natm = mol.natm 
# def generateQ_ij ():
#     q = np.zeros([Natm, occupied, occupied])
#     C = coefficient_matrix ()
#     for atom_id in range (0, Natm):
#         for i in range (0, occupied):
#             for p in range (0, occupied):
#                 for mu in range (0, N_bf):
#                     if AO[mu] == atom_id:
#                         #collect all basis functions centered on atom_id
#                         # the last loop is to sum up all C_mui*C_mup, calculate element q[i,p]
#                         q[atom_id,i,p] += C[mu,i]*C[mu,p]
#                         #q[i,p] += 2*C[i,mu]*C[p,mu]
#     return q

# def generateQ_ab ():
#     q = np.zeros([Natm, virtual, virtual])
#     C = coefficient_matrix ()
#     for atom_id in range (0, Natm):
#         for i in range (0, virtual):
#             for p in range (0, virtual):
#                 for mu in range (0, N_bf):
#                     if AO[mu] == atom_id:
#                         #collect all basis functions centered on atom_id
#                         # the last loop is to sum up all C_mui*C_mup, calculate element q[i,p]
#                         q[atom_id,i,p] += C[mu, occupied + i]*C[mu,occupied + p]
#                         #q[i,p] += 2*C[i,mu]*C[p,mu]
#     return q

# def generateQ_ia ():
#     q = np.zeros([Natm, occupied, virtual])
#     C = coefficient_matrix ()
#     for atom_id in range (0, Natm):
#         for i in range (0, occupied):
#             for p in range (0, virtual):
#                 for mu in range (0, N_bf):
#                     if AO[mu] == atom_id:
#                         #collect all basis functions centered on atom_id
#                         # the last loop is to sum up all C_mui*C_mup, calculate element q[i,p]
#                         q[atom_id,i,p] += C[mu,i]*C[mu, occupied + p]
#                         #q[i,p] += 2*C[i,mu]*C[p,mu]
#     return q

# start = time.time()
# q_tensor_ij = generateQ_ij ()
# q_tensor_ab = generateQ_ab ()
# q_tensor_ia = generateQ_ia ()

# end = time.time()
# print (end - start)

In [ ]:
# plt.matshow(A)
# plt.show()

In [ ]:
# eigv,eigk = np.linalg.eigh(A)
# idx = eigv.argsort()
# eigv = eigv[idx]    #eigenvalues
# eigk = eigk[:,idx]          #eigenkets, m*m

# #np.linalg.eigh guarantees you that the eigenvalues are sorted and uses a faster algorithm 
# #that takes advantage of the fact that the matrix is symmetric. 

# print (np.round (eigv[:10]*27.21138624598853,4))   
# # ':n', first n elements; 'n:' all elements except firt n 


#methanol
# a_x = 0.38
# beta1= 1.86
# beta2=0
# alpha1= 0.9
# alpha2=0
#[12.2388 13.5994 13.7915 15.0407 15.1073 15.245  16.4534 16.5181 16.8506 17.3015]

In [ ]:
td = tddft.TDA(mf)
# td.nstates = 5
# td.conv_tol = 1e-13
# td.kernel()

In [ ]:
# energies, vectors = np.linalg.eigh(H)
# print (energies[:5]*27.21138624598853)

In [ ]:
sTDA_A = build_sTDA_A()
## prepare sTDA_A matrix
vind, hdiag = td.gen_vind(mf)
n = len(hdiag)
I = np.eye(n)

In [ ]:
# H2O Excited State energies (eV)
# [ 5.12051837  7.78183026  8.43305887  9.81364248 10.41186311]
# [ 5.12051842  7.78183032  8.43305894  9.81364256 10.4118632 ]
#


#CH3OH Excited State energies (eV)
#[ 8.73907144 10.77713239 11.26755041 12.203639   12.27658757]
#[ 8.73907152 10.77713248 11.2675505  12.2036391  12.27658768]



# CH3OH
# (351, 351)
# sTDA_A matrix built
# sTDA A matrix as initial guess
# sTDA A matrix as preconditioner
# Iteration steps = 11
# Davidson time: 0.3732
# Excitation_energies =
# [ 8.73907152 10.77713248 11.2675505  12.2036391 ]

# (351, 351)
# sTDA_A matrix built
# sTDA A matrix as initial guess
# Diagonal of full A matrix as preconditioner
# Iteration steps = 15
# Davidson time: 0.3009
# Excitation_energies =
# [ 8.73907152 10.77713248 11.2675505  12.2036391 ]


# (351, 351)
# sTDA_A matrix built
# Diagonal of full A matrix as initial guess
# Diagonal of full A matrix as preconditioner
# Iteration steps = 15
# Davidson time: 0.3152
# Excitation_energies =
# [ 8.73907152 10.77713248 11.2675505  12.2036391 ]


# (351, 351)
# sTDA_A matrix built
# Diagonal of full A matrix as initial guess
# sTDA A matrix as preconditioner
# Iteration steps = 11
# Davidson time: 0.334
# Excitation_energies =
# [ 8.73907152 10.77713248 11.2675505  12.2036391 ]

In [ ]:
# def davidson_A_matrix_hstack (A, k): # matrix A and how many eignvalues to solve
    
#     start = time.time()

#     tol = 1e-5      # Convergence tolerance
#     max = 40      # Maximum number of iterations

#     ###########################################################################################
#     # Begin iterations
#     for i in range(0, max):
#         sum_convec = 0
#         #total converged eigenvectors
#         # if sum_convec == k, break
        
#         lasit_newvec = 0
#         # it records amount of new vectors added in last iteration, ranging from 1 to k
#         # because not all new guess_vectors can survive the Gram-Schmidt
        

#         #################################################
#         # generate initial guess
#         if i == 0:
#             #initial guess  
#             n = np.shape(A)[0]
#             m = k  
#             # m is size of subspace Hamiltonian, amount of initial guesses   
#             # m=k works for H2, m=4k works for H2O

#             V = np.zeros((n, m)) #array of zeros, a container to hold current guess vectors
            
#             sort = np.diag(A).argsort()
#             for j in range(0,m):
#                 V[int(np.argwhere(sort == j)), j] = 1   
#                 # positions with lowest values set as 1
#             W = np.dot(A,V)   
#             # W = Av, create transformed guess vectors
#         #################################################
        
        
#         sub_A = np.dot(V.T, W)  
#         # sub_A is subspace A matrix
#         sub_eigenvalue, sub_eigenket = np.linalg.eigh(sub_A) 
#         # Diagonalize the subspace Hamiltonian, and sorted.
        
        
#         ####################################################################################
#         for x in range(0,k):      
#             #looking at first k vecrors one by one, check if they are roots
#             residual = np.dot((W[:,:m]- sub_eiegnvalue[x]*V[:,:m]), sub_eigenket[:,x])
#             # np.dotV([:,:m])s[:,x]) can transform the subspace-eigenket back into full space eigenket
            
#             norm = np.linalg.norm(residual)
#             if norm <= tol:
#                 sum_convec += 1
#             else:
#                 #print ('norm > tol')
#                 # current guess is not good enough, 
#                 # so we use current guess to create new guess vectors
#                 d = np.diag(A)-sub_eiegnvalue[x]
#                 d[(d<1e-16)&(d>=0)] = 1e-16
#                 d[(d>-1e-16)&(d<0)] = -1e-16   
#                 # kick out all small values
#                 new_vec = residual/d          
#                 # preconditioner
                
#                 new_vec = new_vec/np.linalg.norm (new_vec) 
#                 # normalize before Gram-Schmidt 

#                 for y in range (0, m + lasit_newvec):  
#                     # orthonormalize the new vector against all old vectors
#                     new_vec = new_vec - np.dot(V[:,y], new_vec) * V[:,y]   
                    
#                 norm = np.linalg.norm (new_vec)
#                 if norm > 1e-16:
#                     new_vec = new_vec/norm
#                     # normalzie the new vector, now Gram-Schmidt is done
                    
#                     V = np.append (V, new_vec[:, None], axis=1)
#                     # put the new guess into container
                    
#                     W = np.append (W, np.dot(A, new_vec)[:, None], axis = 1)
#                     # put transformed guess Av into container
                    
#                     lasit_newvec += 1
#         ####################################################################################      
#         if sum_convec == k:
#             break
#         m += lasit_newvec
#     ########################################################################################### 
#     print ('Iteration steps =', i+1)

#     end = time.time()
#     Eigenkets = np.dot(V[:,:m], sub_eigenket[:, :k])
#     print ('Davidson time (no vind function):', round(end-start,4))
#     # return (sub_eiegnvalue[:k], Eigenkets[:,:k])
#     return (sub_eiegnvalue[:k]*27.21138624598853)

# #print (davidson_A_matrix_hstack (A, 5))

In [ ]:
#nocc = mol.nelectron 
# mol.nelectron is number of electrons